## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/hiyab/Desktop/class/module 6 APIs/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iztapa,GT,13.9333,-90.7075,75.69,80,0,4.90,clear sky
1,1,Umm Kaddadah,SD,13.6017,26.6876,58.78,26,3,17.52,clear sky
2,2,Viedma,AR,-40.8135,-62.9967,57.18,45,10,7.70,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,37.06,81,75,14.97,broken clouds
4,4,Gravdal,NO,68.1214,13.5031,34.18,61,94,24.32,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iztapa,GT,13.9333,-90.7075,75.69,80,0,4.90,clear sky
1,1,Umm Kaddadah,SD,13.6017,26.6876,58.78,26,3,17.52,clear sky
2,2,Viedma,AR,-40.8135,-62.9967,57.18,45,10,7.70,clear sky
5,5,Atuona,PF,-9.8000,-139.0333,78.84,78,19,21.39,few clouds
6,6,Ribeira Grande,PT,38.5167,-28.7000,56.77,57,62,15.61,broken clouds


In [15]:
print(len(preferred_cities_df))

399


In [16]:
# 4a. Determine if there are any empty rows.
# preferred_cities_df.count()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

393


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Iztapa,GT,13.9333,-90.7075,75.69,80,0,4.90,clear sky
1,1,Umm Kaddadah,SD,13.6017,26.6876,58.78,26,3,17.52,clear sky
2,2,Viedma,AR,-40.8135,-62.9967,57.18,45,10,7.70,clear sky
5,5,Atuona,PF,-9.8000,-139.0333,78.84,78,19,21.39,few clouds
6,6,Ribeira Grande,PT,38.5167,-28.7000,56.77,57,62,15.61,broken clouds


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Iztapa,GT,75.69,clear sky,13.9333,-90.7075,
1,Umm Kaddadah,SD,58.78,clear sky,13.6017,26.6876,
2,Viedma,AR,57.18,clear sky,-40.8135,-62.9967,
5,Atuona,PF,78.84,few clouds,-9.8000,-139.0333,
6,Ribeira Grande,PT,56.77,broken clouds,38.5167,-28.7000,
7,Castro,BR,63.72,overcast clouds,-24.7911,-50.0119,
9,Busselton,AU,81.05,clear sky,-33.6500,115.3333,
10,Rikitea,PF,77.49,scattered clouds,-23.1203,-134.9692,
11,Coquimbo,CL,62.33,clear sky,-29.9533,-71.3436,
13,Puerto Escondido,MX,77.83,few clouds,15.8500,-97.0667,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {hotel_df.loc[index, 'City']}, {hotel_df.loc[index, 'Country']}... skipping.")        


Hotel not found in Umm Kaddadah, SD... skipping.
Hotel not found in Saint-Louis, SN... skipping.
Hotel not found in Caruray, PH... skipping.
Hotel not found in Airai, TL... skipping.
Hotel not found in Carnot, CF... skipping.
Hotel not found in Taoudenni, ML... skipping.
Hotel not found in Umm Bab, QA... skipping.
Hotel not found in Goure, NE... skipping.
Hotel not found in Bouca, CF... skipping.
Hotel not found in Yangambi, CD... skipping.
Hotel not found in Adrar, MR... skipping.
Hotel not found in Dire, ML... skipping.
Hotel not found in Fonte Boa, BR... skipping.
Hotel not found in Businga, CD... skipping.
Hotel not found in Jocon, HN... skipping.
Hotel not found in Simbahan, PH... skipping.
Hotel not found in Ligayan, PH... skipping.
Hotel not found in Ouadda, CF... skipping.
Hotel not found in Padre Bernardo, BR... skipping.
Hotel not found in Gamba, GA... skipping.
Hotel not found in Araouane, ML... skipping.
Hotel not found in Ouallam, NE... skipping.
Hotel not found in Panguna

In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Iztapa,GT,75.69,clear sky,13.9333,-90.7075,Villas de Puerto Viejo
1,Umm Kaddadah,SD,58.78,clear sky,13.6017,26.6876,
2,Viedma,AR,57.18,clear sky,-40.8135,-62.9967,Hotel Cristal
5,Atuona,PF,78.84,few clouds,-9.8000,-139.0333,Villa Enata
6,Ribeira Grande,PT,56.77,broken clouds,38.5167,-28.7000,Quinta da Meia Eira


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))